In [1]:
from torchvision.datasets import MNIST
from torchvision import transforms
import os

train_data = MNIST(
    root=os.path.join("dataset", "mnist"),
    train=True,
    download=True,
    transform=transforms.ToTensor(),
)
test_data = MNIST(
    root=os.path.join("dataset", "mnist"),
    train=False,
    download=True,
    transform=transforms.ToTensor(),
)


/home/minhyuk/miniconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
import torch
from torch import nn
from collections import OrderedDict

class Model(nn.Module):
    def __init__(self, in_features, out_features=10) -> None:
        super().__init__()
        self.layer1 = nn.Linear(in_features=in_features, out_features=64)
        self.layer2 = nn.Linear(in_features=64, out_features=64)
        self.layer3 = nn.Linear(in_features=64, out_features=64)
        self.layer4 = nn.Linear(in_features=64, out_features=10)

    def forward(self, x: torch.Tensor):
        x = self.layer1(x)
        x = nn.functional.relu(x)
        x = self.layer2(x)
        x = nn.functional.relu(x)
        x = self.layer3(x)
        x = nn.functional.relu(x)
        x = self.layer4(x)
        return x


model = Model(28 * 28).to(device="cuda")
model


Model(
  (layer1): Linear(in_features=784, out_features=64, bias=True)
  (layer2): Linear(in_features=64, out_features=64, bias=True)
  (layer3): Linear(in_features=64, out_features=64, bias=True)
  (layer4): Linear(in_features=64, out_features=10, bias=True)
)

In [17]:
from torch import optim
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


In [18]:
from torch.utils.data import DataLoader

data_loader = DataLoader(
    dataset=train_data, batch_size=32, shuffle=True, drop_last=True
)


In [20]:
epochs = 10
for epoch in range(1, epochs + 1):
    for X, y in data_loader:
        transformed_train_data = (
            X.view(-1, 28 * 28).to(device="cuda")
        )
        train_label = y.to(device="cuda")

        train_preds = model(transformed_train_data)
        loss = criterion(train_preds, train_label)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"epoch {epoch:3}: {loss.item()}")


epoch   1: 2.835209608078003
epoch   2: 2.6049752235412598
epoch   3: 2.3359713554382324
epoch   4: 2.3012800216674805
epoch   5: 2.307130813598633
epoch   6: 2.2725071907043457


KeyboardInterrupt: 

In [8]:
from sklearn.metrics import classification_report

train_preds = model.forward(transformed_train_data)
print(classification_report(
    train_preds.to(device="cpu").detach().numpy().argmax(axis=1),
    train_label.to(device="cpu").numpy(),
))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       1.00      0.12      0.22        32
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0

    accuracy                           0.12        32
   macro avg       0.10      0.01      0.02        32
weighted avg       1.00      0.12      0.22        32



/home/minhyuk/miniconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/minhyuk/miniconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/minhyuk/miniconda3/envs/pytorch/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html